<a href="https://colab.research.google.com/github/magress/MRC/blob/master/Resonet_machine__comprehension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Fri Sep 25 08:21:10 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/magress/Resonet \
&& cd Resonet \
&& git checkout cf57644f28dc20024d87314d190a33b00ee01d00

Cloning into 'Resonet'...
remote: Enumerating objects: 446, done.
remote: Counting objects: 100% (446/446), done.
remote: Compressing objects: 100% (333/333), done.
remote: Total 446 (delta 78), reused 430 (delta 72), pack-reused 0
Receiving objects: 100% (446/446), 2.59 MiB | 16.59 MiB/s, done.
Resolving deltas: 100% (78/78), done.
Note: checking out 'cf57644f28dc20024d87314d190a33b00ee01d00'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at cf57644 First commit


In [ ]:
!pip install ./Resonet

Processing ./Resonet
     |████████████████████████████████| 3.8MB 4.5MB/s 
     |████████████████████████████████| 1.1MB 56.5MB/s 
     |████████████████████████████████| 890kB 58.2MB/s 
  Created wheel for transformers: filename=transformers-2.5.0-cp36-none-any.whl size=481190 sha256=6df924f2e1c9647de2b3d3d5a07af1e6be3ea7062f3edff3c247104519a76514
  Stored in directory: /tmp/pip-ephem-wheel-cache-ocbzq5yv/wheels/17/e3/0a/79c17f86cdf5a5c8b66143501f94a1cd6a086c09a1d15e51dd
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=850ef6e41ce51438d0d6a45d8053db02abb65c32c6256d7bb7142b722a40e85c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built transformers sacremoses


In [ ]:
!pip install tensorboardX

     |████████████████████████████████| 317kB 4.4MB/s 


In [ ]:
import os
import torch
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import (
    AlbertConfig,
    AlbertForQuestionAnswering,
    AlbertTokenizer,
    squad_convert_examples_to_features
)

from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample

from transformers.data.metrics.squad_metrics import compute_predictions_logits

# READER NOTE: Set this flag to use own model, or use pretrained model in the Hugging Face repository
use_own_model = False

if use_own_model:
  model_name_or_path = "/content/model_output"
else:
  model_name_or_path = "ktrapeznikov/albert-xlarge-v2-squad-v2"

output_dir = ""

# Config
n_best_size = 1
max_answer_length = 30
do_lower_case = True
null_score_diff_threshold = 0.0

def to_list(tensor):
    return tensor.detach().cpu().tolist()

# Setup model
config_class, model_class, tokenizer_class = (
    AlbertConfig, AlbertForQuestionAnswering, AlbertTokenizer)
config = config_class.from_pretrained(model_name_or_path)
tokenizer = tokenizer_class.from_pretrained(
    model_name_or_path, do_lower_case=True)
model = model_class.from_pretrained(model_name_or_path, config=config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

processor = SquadV2Processor()

def run_prediction(question_texts, context_text):
    """Setup function to compute predictions"""
    examples = []

    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            is_impossible=False,
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=384,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    output_prediction_file = "predictions.json"
    output_nbest_file = "nbest_predictions.json"
    output_null_log_odds_file = "null_predictions.json"

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,  # verbose_logging
        True,  # version_2_with_negative
        null_score_diff_threshold,
        tokenizer,
    )

    return predictions

In [ ]:

context="Akbar was born on October 1542, he was the third Mughal emperor successor, who reigned from 1556 to 1605. Akbar succeeded his father, Humayun, under a regent, Bairam Khan, who helped the young emperor expand and consolidate Mughal domains in India. He died on 27 October 1605. Akbar ascended the Mughal throne at the age of 13. Akbar's contribution to India was to developed a strong and stable economy, leading to commercial expansion and greater patronage of culture. Akbar was succeeded as emperor by his son, Prince Salim."
questions = ["who was Bairam Khan?"]

# Run method
predictions = run_prediction(questions, context)
# Print results
for key in predictions.keys():
  if (predictions[key]==""):
    print("No answer found")
  else:
    print(predictions[key])

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1384.26it/s]


regent


In [ ]:
# from google.colab import files
# uploaded = files.upload()
#  !pip install tika 
#  from tika import parser

#  file = 'mrcfinal.pdf'
 
#  file_data = parser.from_file(file)
#  #Get files text content
#  text = file_data['content']
#  context= " ".join(text.split())
#  print(context)